In [4]:
# import argparse

class HParams(object):
	def __init__(self):
		# Dataset Settings
		self.dataset_path = 'd:/다운로드/gtzan'
		self.feature_path = 'feature5'
		self.genres = ['classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae']

		# Feature Parameters
		self.sample_rate = 22050
		self.fft_size = 1024
		self.win_size = 1024
		self.hop_size = 512
		self.num_mels = 128
		self.feature_length = 1024  # audio length = feature_length*hop_size/sample_rate (s)

		# Training Parameters
		self.device = 1  # 0: CPU, 1: GPU0, 2: GPU1, ...
		self.batch_size = 20
		self.num_epochs = 100
		self.learning_rate = 3*1e-5
		self.stopping_rate = 1e-12
		self.weight_decay = 1e-6
		self.momentum = 0.9
		self.factor = 0.2
		self.patience = 3.

hparams = HParams()

In [5]:
import os
import numpy as np
from torch.utils.data import Dataset, DataLoader

class GTZANDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.x.shape[0]

# Function to get genre index for the give file
def get_label(file_name, hparams):
    genre = file_name.split('.')[0]
    label = hparams.genres.index(genre)
    return label

def load_dataset(set_name, hparams, part=2): #part 1홀수만, part 0짝수만, part 2,3,4... 모두
    x = []
    y = []

    dataset_path = os.path.join(hparams.feature_path, set_name)
    
    if part == 0 :
        i=0
        for root,dirs,files in os.walk(dataset_path):
            for file in files:
                if i==0:
                    i+=1
                else:
                    data = np.load(os.path.join(root,file))
                    label = get_label(file, hparams)
                    x.append(data)
                    y.append(label)
    
    elif part == 1 :
        i=0
        for root,dirs,files in os.walk(dataset_path):
            for file in files:
                if i==0:
                    data = np.load(os.path.join(root,file))
                    label = get_label(file, hparams)
                    x.append(data)
                    y.append(label)
                    i+=1
                else :
                    i=0
    else :
        for root,dirs,files in os.walk(dataset_path):
            for file in files:
                data = np.load(os.path.join(root,file))
                label = get_label(file, hparams)
                x.append(data)
                y.append(label)


    x = np.stack(x)
    y = np.stack(y)

    return x,y



In [12]:
load_dataset('valid', hparams, 1)[0].shape

(80, 1024, 128)

In [4]:
load_dataset('valid', hparams)[0].shape

(160, 1024, 128)

In [17]:
def get_valid_test_dataloader(hparams):
    x_train, y_train = load_dataset('train', hparams, 1)
    x_valid, y_valid = load_dataset('valid', hparams)
    x_test, y_test = load_dataset('test', hparams)

    mean = np.mean(x_valid)
    std = np.std(x_valid)
    x_train = (x_train - mean)/std
    x_valid = (x_valid - mean)/std
    x_test = (x_test - mean)/std

    train_set = GTZANDataset(x_train, y_train)
    vaild_set = GTZANDataset(x_valid, y_valid)
    test_set = GTZANDataset(x_test, y_test)

    train_loader = DataLoader(train_set, batch_size=hparams.batch_size, shuffle=True, drop_last=False)
    valid_loader = DataLoader(vaild_set, batch_size=hparams.batch_size, shuffle=False, drop_last=False)
    test_loader = DataLoader(test_set, batch_size=hparams.batch_size, shuffle=False, drop_last=False)

    return train_loader, valid_loader, test_loader

In [ ]:
def get_valid_test_dataloader(hparams):
    x_valid, y_valid = load_dataset('valid', hparams)
    x_test, y_test = load_dataset('test', hparams)

    mean = np.mean(x_valid)
    std = np.std(x_valid)
    x_valid = (x_valid - mean)/std
    x_test = (x_test - mean)/std

    vaild_set = GTZANDataset(x_valid, y_valid)
    test_set = GTZANDataset(x_test, y_test)

    valid_loader = DataLoader(vaild_set, batch_size=hparams.batch_size, shuffle=False, drop_last=False)
    test_loader = DataLoader(test_set, batch_size=hparams.batch_size, shuffle=False, drop_last=False)

    return valid_loader, test_loader

In [6]:
x_train1, y_train1 = load_dataset('train', hparams,1)

In [8]:
import torch
torch.cuda.empty_cache() 

In [9]:
x_train2, y_train2 = load_dataset('train', hparams,2)

In [10]:
x_train1.shape

(3840, 1024, 128)

In [12]:
x_train2.shape

(7680, 1024, 128)

In [18]:
x_valid, y_valid = load_dataset('valid', hparams)
x_test, y_test = load_dataset('test', hparams)

In [19]:
mean = np.mean(x_valid)
std = np.std(x_valid)
x_valid1 = (x_valid - mean)/std
x_test1 = (x_test - mean)/std

In [20]:
print(mean,std)

0.18346219 0.22429962


In [21]:
np.mean(x_test),np.std(x_test)

(0.18378183, 0.22642404)

array([[[-0.02365023,  0.4747217 ,  0.12502606, ..., -0.81216997,
         -0.8121109 , -0.81203663],
        [ 0.16711591,  0.7383559 ,  0.44936776, ..., -0.8177684 ,
         -0.8177805 , -0.81778395],
        [-0.0544921 ,  0.3997801 ,  0.6830997 , ..., -0.81776077,
         -0.8178003 , -0.81778353],
        ...,
        [-0.5111277 , -0.67734456, -0.7066235 , ..., -0.81779426,
         -0.81777304, -0.8177853 ],
        [-0.43083227, -0.48608267, -0.6176315 , ..., -0.8177912 ,
         -0.81779164, -0.8178385 ],
        [-0.50620425, -0.66575384, -0.7546493 , ..., -0.81779706,
         -0.8177804 , -0.8177574 ]],

       [[ 1.1686407 ,  1.4975377 ,  1.7082962 , ..., -0.79947305,
         -0.7999146 , -0.8001944 ],
        [-0.39733228,  0.3581721 ,  2.144388  , ..., -0.8176981 ,
         -0.81777656, -0.8178265 ],
        [-0.18996601,  0.7580676 ,  2.089091  , ..., -0.8176981 ,
         -0.8177939 , -0.8177847 ],
        ...,
        [ 0.8363309 ,  1.5139589 ,  1.427164  , ..., -

In [11]:
train_loader, valid_loader, test_loader = get_dataloader(hparams)

ValueError: not enough values to unpack (expected 3, got 2)

In [10]:
valid_loader, test_loader = get_valid_test_dataloader(hparams

In [ ]:
'''
train_test.py
A file for training model for genre classification.
Please check the device in hparams.py before you run this code.
'''
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from matplotlib import pyplot as plt

# import data_manager
# import models
# from hparams import hparams

# Wrapper class to run PyTorch model
class Runner(object):
    def __init__(self, hparams):
        self.model = Baseline(hparams)
        self.criterion = nn.CrossEntropyLoss()
        
        #self.optimizer = torch.optim.Adam(self.model.parameters(), lr=hparams.learning_rate)
        self.optimizer = torch.optim.SGD(self.model.parameters(), lr=hparams.learning_rate, momentum=hparams.momentum)
        self.scheduler = ReduceLROnPlateau(self.optimizer, mode='min', factor=hparams.factor, patience=hparams.patience, verbose=True)
        self.learning_rate = hparams.learning_rate
        self.stopping_rate = hparams.stopping_rate
        self.device = torch.device("cpu")

        if hparams.device > 0:
            torch.cuda.set_device(hparams.device - 1)##
            self.model.cuda(hparams.device - 1)
            self.criterion.cuda(hparams.device - 1)
            self.device = torch.device("cuda:" + str(hparams.device - 1))
            
#         device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#         self.model = Baseline(hparams).to(device)
#         if torch.cuda.device_count() > 1:
#             print('\n===> Training on GPU!')
#             self.model = nn.DataParallel(net)

    # Accuracy function works like loss function in PyTorch
    def accuracy(self, source, target):
        source = source.max(1)[1].long().cpu()
        target = target.cpu()
        correct = (source == target).sum().item()

        return correct/float(source.size(0))

    # Running model for train, test and validation. mode: 'train' for training, 'eval' for validation and test
    def run(self, dataloader, mode='train'):
        self.model.train() if mode is 'train' else self.model.eval()

        epoch_loss = 0.0
        epoch_acc = 0.0
        
        for batch, (x, y) in enumerate(dataloader):
            x = x.to(self.device)
            y = y.to(self.device).type(torch.long)
            
            #print(y)

            prediction = self.model(x)
            loss = self.criterion(prediction, y)
            acc = self.accuracy(prediction, y)

            if mode is 'train':
                loss.backward()
                self.optimizer.step()
                self.optimizer.zero_grad()

            epoch_loss += prediction.size(0)*loss.item()
            epoch_acc += prediction.size(0)*acc

        epoch_loss = epoch_loss/len(dataloader.dataset)
        epoch_acc = epoch_acc/len(dataloader.dataset)

        return epoch_loss, epoch_acc

    # Early stopping function for given validation loss
    def early_stop(self, loss, epoch):
        self.scheduler.step(loss, epoch)
        self.learning_rate = self.optimizer.param_groups[0]['lr']
        stop = self.learning_rate < self.stopping_rate

        return stop

def device_name(device):
    if device == 0:
        device_name = 'CPU'
    else:
        device_name = 'GPU:' + str(device - 1)

    return device_name

def main_1():  # 4분할 하여 처리
    runner = Runner(hparams)
    val_loss_ls =[]
    train_loss_ls =[]

    print('Training on ' + device_name(hparams.device))
    for epoch in range(25):
        train_loss, train_acc = runner.run(train_loader, 'train')
        valid_loss, valid_acc = runner.run(valid_loader, 'eval')
        if((epoch+1)%10==0):
            print("[Epoch %d/%d] [Train Loss: %.4f] [Train Acc: %.4f] [Valid Loss: %.4f] [Valid Acc: %.4f]" %
                  (epoch + 1, hparams.num_epochs, train_loss, train_acc, valid_loss, valid_acc))
            
        val_loss_ls.append(valid_loss)
        train_loss_ls.append(train_loss)
        
        if runner.early_stop(valid_loss, epoch + 1):
            break

    test_loss, test_acc = runner.run(test_loader, 'eval')
    print("Training Finished")
    print("Test Accuracy: %.2f%%" % (100*test_acc))
    
    plt.plot(val_loss_ls,'r')
    plt.plot(train_loss_ls,'b')
    plt.show()
    

In [14]:
l = range(10,100)

In [15]:
l

range(10, 100)

In [16]:
for i in l:
    print(i)

10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [2]:
import sys

#사람이 보기 편함

sys.version

'3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)]'